In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn import model_selection, preprocessing, linear_model
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'


import numpy as np
import re
import io
import time

C:\Users\snetkova\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\snetkova\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\snetkova\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
gpu = gpus[0]
tf.config.experimental.set_memory_growth(gpu, True)

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping 

from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, MaxPooling1D, SimpleRNN, LSTM, GRU

In [3]:
import zipfile
with zipfile.ZipFile('D:/GeekBrains/nlp/les10/rus-eng.zip', 'r') as zip_ref:
    zip_ref.extractall('')

In [4]:
path_to_file = "rus.txt"

#### preprocess data

In [5]:
def preprocess_sentence(w):
    w = w.lower().strip()

    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w) #collapse whitespaces

    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Zа-яА-Я?.!,']+", " ", w)

    w = w.strip() #Remove spaces at the beginning and at the end of the string

    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    return w

In [6]:
preprocess_sentence("I can't go.")

"<start> i can't go . <end>"

#### create dataset

In [7]:
# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [ENG, RUS]
def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')[:2]]  for l in lines[:num_examples]]

    return zip(*word_pairs)

In [8]:
en, ru = create_dataset(path_to_file, None)
print(en[0])
print(ru[0])

<start> go . <end>
<start> марш ! <end>


In [9]:
en[:4]

('<start> go . <end>',
 '<start> go . <end>',
 '<start> go . <end>',
 '<start> hi . <end>')

In [10]:
len(en), len(ru)

(431097, 431097)

In [11]:
def tokenize(lang):
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
    lang_tokenizer.fit_on_texts(lang)

    tensor = lang_tokenizer.texts_to_sequences(lang)

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

    return tensor, lang_tokenizer

In [12]:
tokenize(en[:2])

(array([[1, 2, 3, 4],
        [1, 2, 3, 4]]),
 <keras_preprocessing.text.Tokenizer at 0x1e787dead88>)

In [13]:
tokenize(en[:10])[0].shape

(10, 4)

In [14]:
tokenize(en)[0].shape

(431097, 112)

In [15]:
def load_dataset(path, num_examples=None):
    # creating cleaned input, output pairs
#     targ_lang, inp_lang = create_dataset(path, num_examples) # ru -> en
    targ_lang, inp_lang = create_dataset(path, num_examples) # en -> ru
    
    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

    return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [16]:
# Try experimenting with the size of that dataset
# num_examples = len(en)
num_examples = 150000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = target_tensor.shape[1], input_tensor.shape[1]

In [17]:
max_length_targ, max_length_inp

(12, 15)

In [18]:
input_tensor.shape

(150000, 15)

In [19]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

120000 120000 30000 30000


In [20]:
def convert(lang, tensor):
    for t in tensor:
        if t!=0:
            print ("%d ----> %s" % (t, lang.index_word[t]))

In [21]:
input_tensor_train[0]

array([  1,   4,  92, 216, 100,   3,   2,   0,   0,   0,   0,   0,   0,
         0,   0])

In [22]:
convert(inp_lang, input_tensor_train[0])

1 ----> <start>
4 ----> я
92 ----> хотел
216 ----> уйти
100 ----> домой
3 ----> .
2 ----> <end>


In [23]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 128

steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 300
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [24]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([128, 15]), TensorShape([128, 12]))

#### model

In [25]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=False,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')


    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        return state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [26]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_hidden = encoder.call(example_input_batch, sample_hidden)
# print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

Encoder Hidden state shape: (batch size, units) (128, 1024)


In [27]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, x, hidden):
        # enc_output shape == (batch_size, max_length, hidden_size)

        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)

        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)

        # passing the concatenated vector to the GRU
        output, state = self.gru(x, initial_state=hidden)

        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))

        # output shape == (batch_size, vocab)
        x = self.fc(output)

        return x, state

In [28]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)
decoder_sample_x, decoder_sample_h = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden)
decoder_sample_x.shape, decoder_sample_h.shape

(TensorShape([128, 8879]), TensorShape([128, 1024]))

In [29]:
optimizer = tf.keras.optimizers.Adam()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [30]:
checkpoint_dir = './training_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")

checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [31]:
@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0

    with tf.GradientTape() as tape:
        enc_hidden = encoder(inp, enc_hidden)

        dec_hidden = enc_hidden

        dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

        # Teacher forcing - feeding the target as the next input
        for t in range(1, targ.shape[1]):
            # passing enc_output to the decoder
            predictions, dec_hidden = decoder(dec_input, dec_hidden)

            loss += loss_function(targ[:, t], predictions)

            # using teacher forcing
            dec_input = tf.expand_dims(targ[:, t], 1)

    batch_loss = (loss / int(targ.shape[1]))

    variables = encoder.trainable_variables + decoder.trainable_variables

    gradients = tape.gradient(loss, variables)

    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

In [32]:
# EPOCHS = 50

# for epoch in range(EPOCHS):
#     start = time.time()

#     enc_hidden = encoder.initialize_hidden_state()
#     total_loss = 0

#     for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
#         batch_loss = train_step(inp, targ, enc_hidden)
#         total_loss += batch_loss

#         if batch % 100 == 0:
#             print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
#                                                        batch,
#                                                        batch_loss.numpy()))
#     # saving (checkpoint) the model every 2 epochs
#     if (epoch + 1) % 10 == 0:
#         checkpoint.save(file_prefix = checkpoint_prefix)

#     print('Epoch {} Loss {:.4f}'.format(epoch + 1,
#                                       total_loss / steps_per_epoch))
#     print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 4.6458
Epoch 1 Batch 100 Loss 2.0260
Epoch 1 Batch 200 Loss 1.7946
Epoch 1 Batch 300 Loss 1.6387
Epoch 1 Batch 400 Loss 1.5264
Epoch 1 Batch 500 Loss 1.4707
Epoch 1 Batch 600 Loss 1.3485
Epoch 1 Batch 700 Loss 1.3627
Epoch 1 Batch 800 Loss 1.1597
Epoch 1 Batch 900 Loss 1.1853
Epoch 1 Loss 1.5698
Time taken for 1 epoch 198.24812364578247 sec

Epoch 2 Batch 0 Loss 1.0496
Epoch 2 Batch 100 Loss 0.9514
Epoch 2 Batch 200 Loss 0.9217
Epoch 2 Batch 300 Loss 0.9263
Epoch 2 Batch 400 Loss 0.8791
Epoch 2 Batch 500 Loss 0.8178
Epoch 2 Batch 600 Loss 0.7389
Epoch 2 Batch 700 Loss 0.7312
Epoch 2 Batch 800 Loss 0.6900
Epoch 2 Batch 900 Loss 0.7538
Epoch 2 Loss 0.8216
Time taken for 1 epoch 183.61669516563416 sec

Epoch 3 Batch 0 Loss 0.5088
Epoch 3 Batch 100 Loss 0.4935
Epoch 3 Batch 200 Loss 0.4768
Epoch 3 Batch 300 Loss 0.4721
Epoch 3 Batch 400 Loss 0.5239
Epoch 3 Batch 500 Loss 0.4420
Epoch 3 Batch 600 Loss 0.4003
Epoch 3 Batch 700 Loss 0.4664
Epoch 3 Batch 800 Loss 0.3937
Ep

Epoch 23 Batch 100 Loss 0.0331
Epoch 23 Batch 200 Loss 0.0531
Epoch 23 Batch 300 Loss 0.0392
Epoch 23 Batch 400 Loss 0.0549
Epoch 23 Batch 500 Loss 0.0405
Epoch 23 Batch 600 Loss 0.0609
Epoch 23 Batch 700 Loss 0.0553
Epoch 23 Batch 800 Loss 0.0506
Epoch 23 Batch 900 Loss 0.0745
Epoch 23 Loss 0.0574
Time taken for 1 epoch 185.36056971549988 sec

Epoch 24 Batch 0 Loss 0.0494
Epoch 24 Batch 100 Loss 0.0494
Epoch 24 Batch 200 Loss 0.0367
Epoch 24 Batch 300 Loss 0.0705
Epoch 24 Batch 400 Loss 0.0667
Epoch 24 Batch 500 Loss 0.0648
Epoch 24 Batch 600 Loss 0.0579
Epoch 24 Batch 700 Loss 0.0502
Epoch 24 Batch 800 Loss 0.0905
Epoch 24 Batch 900 Loss 0.0683
Epoch 24 Loss 0.0569
Time taken for 1 epoch 185.57644057273865 sec

Epoch 25 Batch 0 Loss 0.0418
Epoch 25 Batch 100 Loss 0.0387
Epoch 25 Batch 200 Loss 0.0483
Epoch 25 Batch 300 Loss 0.0608
Epoch 25 Batch 400 Loss 0.0676
Epoch 25 Batch 500 Loss 0.0539
Epoch 25 Batch 600 Loss 0.0694
Epoch 25 Batch 700 Loss 0.0640
Epoch 25 Batch 800 Loss 0.0566


Epoch 45 Batch 0 Loss 0.0357
Epoch 45 Batch 100 Loss 0.0374
Epoch 45 Batch 200 Loss 0.0398
Epoch 45 Batch 300 Loss 0.0486
Epoch 45 Batch 400 Loss 0.0367
Epoch 45 Batch 500 Loss 0.0431
Epoch 45 Batch 600 Loss 0.0426
Epoch 45 Batch 700 Loss 0.0492
Epoch 45 Batch 800 Loss 0.0555
Epoch 45 Batch 900 Loss 0.0722
Epoch 45 Loss 0.0477
Time taken for 1 epoch 182.11389183998108 sec

Epoch 46 Batch 0 Loss 0.0357
Epoch 46 Batch 100 Loss 0.0557
Epoch 46 Batch 200 Loss 0.0472
Epoch 46 Batch 300 Loss 0.0515
Epoch 46 Batch 400 Loss 0.0381
Epoch 46 Batch 500 Loss 0.0415
Epoch 46 Batch 600 Loss 0.0739
Epoch 46 Batch 700 Loss 0.0430
Epoch 46 Batch 800 Loss 0.0434
Epoch 46 Batch 900 Loss 0.0665
Epoch 46 Loss 0.0474
Time taken for 1 epoch 182.3259994983673 sec

Epoch 47 Batch 0 Loss 0.0345
Epoch 47 Batch 100 Loss 0.0340
Epoch 47 Batch 200 Loss 0.0299
Epoch 47 Batch 300 Loss 0.0374
Epoch 47 Batch 400 Loss 0.0367
Epoch 47 Batch 500 Loss 0.0503
Epoch 47 Batch 600 Loss 0.0416
Epoch 47 Batch 700 Loss 0.0377
Epo

#### translate

In [33]:
def evaluate(sentence):
    attention_plot = np.zeros((max_length_targ, max_length_inp))

    sentence = preprocess_sentence(sentence)

    inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
    inputs = tf.convert_to_tensor(inputs)

    result = ''

    hidden = [tf.zeros((1, units))]
    enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden = decoder(dec_input, dec_hidden)

        # storing the attention weights to plot later on
        predicted_id = tf.argmax(predictions[0]).numpy()
        result += targ_lang.index_word[predicted_id] + ' '

        if targ_lang.index_word[predicted_id] == '<end>':
            return result, sentence

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence

In [34]:
def translate(sentence):
    result, sentence = evaluate(sentence)

    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

In [35]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [37]:
translate('Началось все с котов.')

Input: <start> началось все с котов . <end>
Predicted translation: there's nobody going out . <end> 


In [39]:
translate('Нужно бежать со всех ног')

Input: <start> нужно бежать со всех ног <end>
Predicted translation: she must contact anybody . <end> 


In [40]:
translate('Нужно бежать со всех ног, чтобы только оставаться на месте, а чтобы куда-то попасть, надо бежать как минимум вдвое быстрее')

Input: <start> нужно бежать со всех ног , чтобы только оставаться на месте , а чтобы куда то попасть , надо бежать как минимум вдвое быстрее <end>
Predicted translation: no way somewhere else . <end> 


Ну, так себе переводчик получился, если честно